In [2]:
import pandas as pd
import numpy
from imblearn.over_sampling import SMOTE
from collections import Counter
from sklearn import model_selection
import xgboost as xgb
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import pickle
from sklearn.model_selection import GridSearchCV


In [3]:
dataFramePenyakitJantung = pd.read_csv("./dataset/heart.csv")

# Tidak ada missing value
# print(f"Missing Value: \n{ dataFramePenyakitJantung.isnull().sum() }")

# duplicate data

# dataFramePenyakitJantung[dataFramePenyakitJantung.duplicated(keep="first")]

dataFramePenyakitJantung.duplicated().sum()
dataFramePenyakitJantung[dataFramePenyakitJantung.duplicated(keep=False)].sort_values(by=list(dataFramePenyakitJantung.columns))
no_dup_data = dataFramePenyakitJantung.drop_duplicates()

no_dup_data


,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,52,1,0,125,212,0,1,168,0,1.0,2,2,3,0
1,53,1,0,140,203,1,0,155,1,3.1,0,0,3,0
2,70,1,0,145,174,0,1,125,1,2.6,0,0,3,0
3,61,1,0,148,203,0,1,161,0,0.0,2,1,3,0
4,62,0,0,138,294,1,1,106,0,1.9,1,3,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
723,68,0,2,120,211,0,0,115,0,1.5,1,0,2,1
733,44,0,2,108,141,0,1,175,0,0.6,1,0,2,1
739,52,1,0,128,255,0,1,161,1,0.0,2,1,3,0
843,59,1,3,160,273,0,0,125,0,0.0,2,0,2,0


In [ ]:
X = no_dup_data.drop(columns="target")
y = no_dup_data['target']

smote = SMOTE(random_state=42)


# Split Data
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.2, random_state=42)

X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

print(f"Data train Sebelum di Oversampling: {X_train.shape[0]} baris")
print(f"Data train Sebelum di Oversampling: {X_resampled.shape[0]} baris")

print()
print("Jumlah Class 0 dan 1, sebelum di oversampling")
print(y_train.value_counts())
print()
print("Jumlah Class 0 dan 1, setalah di oversampling")
print(y_resampled.value_counts())



Data train Sebelum di Oversampling: 241 baris
Data train Sebelum di Oversampling: 270 baris

Jumlah Class 0 dan 1, sebelum di oversampling
target
1    135
0    106
Name: count, dtype: int64

Jumlah Class 0 dan 1, setalah di oversampling
target
0    135
1    135
Name: count, dtype: int64


In [ ]:
param_grid_xgboost = {
    'n_estimators': [600, 1000],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [9, 10, 13],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.7, 0.9, 1.0],
    'gamma': [0.3, 0.5],
    'reg_alpha': [0, 0.001, 0.01, 0.1],
    'reg_lambda': [1, 0.1],
    'min_child_weight': [1, 3, 5],
}

params = {
            'min_child_weight': [1, 5, 10],
            'gamma': [0.5, 1, 1.5, 2, 5],
            'subsample': [0.6, 0.8, 1.0],
            'colsample_bytree': [0.6, 0.8, 1.0],
            'max_depth': [3, 4, 5, 13],
            'n_estimator': [500, 1000],
            'learning_rate': [0.1, 1, 0.002]
        }


model_xgb = xgb.XGBClassifier(
    objective='binary:logistic',
    eval_metric='logloss',
    use_label_encoder=False,
    n_estimators=1000,      # Jumlah pohon
    learning_rate=0.01,     # Laju pembelajaran
    max_depth=13,           # Kedalaman maksimum pohon
    subsample=0.8,         # Fraksi sampel untuk setiap pohon
    colsample_bytree=0.5,  # Fraksi fitur untuk setiap pohon
    gamma=0.5,             # Minimum loss reduction
    random_state=42
)

grid_search = GridSearchCV(
    estimator=model_xgb,
    param_grid=params,
    scoring='precision', # Metrik untuk optimasi
    cv=5,               # Jumlah fold untuk cross-validation
    n_jobs=-1,          # Gunakan semua core CPU
    verbose=1
)

model_xgb.fit(X_train, y_train)

grid_search.fit(X_train, y_train)
best_xgb_model = grid_search.best_estimator_

c:\Python312\Lib\site-packages\xgboost\training.py:183: UserWarning: [19:08:58] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Fitting 5 folds for each of 3240 candidates, totalling 16200 fits


c:\Python312\Lib\site-packages\xgboost\training.py:183: UserWarning: [19:14:10] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "n_estimator", "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


In [42]:
y_pred = model_xgb.predict(X_test)

y_pred_best = best_xgb_model.predict(X_test)

In [43]:
print("model xgb")
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

print("\nClassification Report:")
print(classification_report(y_test, y_pred))

print("Best Model XGB")
accuracy = accuracy_score(y_test, y_pred_best)
print(f"Accuracy: {accuracy:.4f}")

print("\nClassification Report:")
print(classification_report(y_test, y_pred_best))

model xgb
Accuracy: 0.8361

Classification Report:
              precision    recall  f1-score   support

           0       0.89      0.78      0.83        32
           1       0.79      0.90      0.84        29

    accuracy                           0.84        61
   macro avg       0.84      0.84      0.84        61
weighted avg       0.84      0.84      0.84        61

Best Model XGB
Accuracy: 0.8033

Classification Report:
              precision    recall  f1-score   support

           0       0.83      0.78      0.81        32
           1       0.77      0.83      0.80        29

    accuracy                           0.80        61
   macro avg       0.80      0.80      0.80        61
weighted avg       0.81      0.80      0.80        61



In [30]:
nama_model = "klasifikasi_penyakit_jantung_xgboost.pkl"

with open(nama_model, 'wb') as file:
    pickle.dump(best_xgb_model, file)